In [1]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from copy import deepcopy
from math import factorial, exp

In [2]:
import logging

# Retrieve the logger configured in experiment_1.ipynb
logger = logging.getLogger()

In [3]:
%run "attack_graph_MARA.ipynb"

## Add Virtual starting node

In [ ]:
def find_and_add_entry_node(graph):
    # Find all nodes with no incoming edges
    roots = [n for n, deg in graph.in_degree() if deg == 0]
    #print("DEBUG: Roots before adding entry node:", roots)
    
    if len(roots) > 1:
        # Create a virtual entry node
        entry = "attacker_entry_node"
        graph.add_node(entry)
        
        # Connect the entry node to all roots
        for r in roots:
            graph.add_edge(entry, r, weight=1)
        
        # print("DEBUG: Multiple roots found. Created entry node:", entry)
        # print("DEBUG: Edges from entry node to roots:")
        # for r in roots:
        #     print(f"  {entry} -> {r}")
        # return entry, graph
    else:
        # If single root exists, that is our entry
        entry = roots[0]
        #print("DEBUG: Only one root found. Entry node is:", entry)
        return entry, graph


In [5]:
def process_target_nodes(graph):
    # In R:
    # jointVertex <- gorder(attack_graph) - length(target_list) + 1
    # vertexNo[, target_list] <- jointVertex
    # vertexNo[vertexNo == 0] <- 1:(jointVertex - 1)
    # attack_graph <- contract.vertices(attack_graph, mapping = vertexNo)

    # Step 1: Identify target nodes (no outgoing edges)
    targets = [n for n, deg in graph.out_degree() if deg == 0]
    if len(targets) <= 1:
        # If there is 0 or 1 target, no merging needed
        return graph

    # Number of original nodes
    original_count = graph.number_of_nodes()
    # R code formula for the new merged target node ID
    joint_vertex_num = original_count - len(targets) + 1

    # We will mimic exactly the logic from R:
    # First, get a topological order of the original graph
    topo_nodes = list(nx.topological_sort(graph))

    # Create a mapping array: For each node in topo order, assign a "vertexNo".
    # Initialize everything to 0
    vertexNo = {node: 0 for node in topo_nodes}

    # Set all targets to joint_vertex_num
    for t in targets:
        vertexNo[t] = joint_vertex_num

    # Now fill the zeros with consecutive integers from 1 to joint_vertex_num-1
    # but maintain the topological order
    fill_value = 1
    for node in topo_nodes:
        if vertexNo[node] == 0:
            vertexNo[node] = fill_value
            fill_value += 1

    # Now we have a mapping of each original node to an integer ID.
    # Nodes assigned the same integer ID should be contracted into one node.
    # Example: If multiple targets got joint_vertex_num, they form one merged node.

    # Create a mapping from the integer IDs to sets of original nodes
    block_map = {}
    for node, grp_id in vertexNo.items():
        if grp_id not in block_map:
            block_map[grp_id] = set()
        block_map[grp_id].add(node)

    # Build a new contracted graph
    merged_graph = nx.DiGraph()

    # Add nodes (each group_id becomes one node in merged_graph)
    for grp_id in block_map:
        merged_graph.add_node(grp_id)

    # For edges: if u->v in the old graph, map them to their groups:
    # If they map to the same group, skip (self-loop)
    # Otherwise, add edge between the groups
    for u, v, data in graph.edges(data=True):
        u_id = vertexNo[u]
        v_id = vertexNo[v]
        if u_id != v_id:
            # Keep the weight as is
            w = data.get('weight', 1)
            if not merged_graph.has_edge(u_id, v_id):
                merged_graph.add_edge(u_id, v_id, weight=w)

    return merged_graph

In [22]:
# Prepare the graph
work_graph = deepcopy(attack_graph)
entry_node, work_graph = find_and_add_entry_node(work_graph)

# print("DEBUG: Current graph nodes after handling entry node:", list(work_graph.nodes()))
# print("DEBUG: Current graph edges after handling entry node:")
# for u,v in work_graph.edges():
#     print(f"  {u} -> {v}")

work_graph = process_target_nodes(work_graph)

# print("DEBUG: Final graph after processing targets:")
# print("Nodes:", list(work_graph.nodes()))
# print("Edges:")
# for u,v in work_graph.edges():
#     print(f"  {u} -> {v}")

# new_targets = [n for n, deg in work_graph.out_degree() if deg == 0]
# print("DEBUG: New targets after merging:", new_targets)

## Next part

In [7]:
def generate_game_elements(graph, entry_node):
    """
    Generate game elements after we have the final graph (with single entry and single target).
    """
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
        print("WARNING: Expected exactly one target node after contraction. Found:", target_list)
    
    routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))
    V = sorted(list(set([node for path in routes for node in path])))
    topo_all = list(nx.topological_sort(graph))
    node_order = [n for n in topo_all if n in V]
    
    excluded = {entry_node} | set(target_list)
    if entry_node == "attacker_entry_node":
        roots = [r for r, deg in graph.in_degree() if deg == 0 and r != entry_node]
        excluded.update(roots)
    
    as1 = [n for n in V if n not in excluded]
    as2 = routes
    
    return routes, V, as1, as2, target_list, node_order

def setup_game_parameters(V, routes, entry_node, target_list):
    adv_list = [n for n in V if n not in [entry_node] + target_list]
    if len(adv_list) == 0:
        print("WARNING: No adversary intermediate locations found. Check graph structure.")
    
    theta = {loc: 1/len(adv_list) for loc in adv_list}
    m = len(routes)
    return adv_list, theta, m

def lossDistribution(U):
    U = U / np.sum(U)
    support = np.arange(1, len(U)+1)
    dpdf = U
    cdf = np.cumsum(U)
    tail = 1 - cdf + U
    return {
        'support': support,
        'dpdf': dpdf,
        'cdf': cdf,
        'tail': tail
    }

def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                  attack_rate, defense_rate, node_order):
    payoffs = []
    for i in as1:
        for path in as2:
            U = np.zeros(len(V))
            for avatar in adv_list:
                L = np.zeros(len(V))
                if avatar in path:
                    start_idx = path.index(avatar)
                    route = path[start_idx:]
                    pdf_d = random_steps_fn(route, attack_rate, defense_rate)
                    
                    if i in route:
                        cutPoint = route.index(i) + 1
                    else:
                        cutPoint = len(route)
                    
                    if np.sum(pdf_d[:cutPoint]) == 0:
                        payoffDistr = np.zeros(cutPoint)
                        payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_d[:cutPoint] / np.sum(pdf_d[:cutPoint])
                    
                    for idx, node in enumerate(route[:cutPoint]):
                        L[V.index(node)] = payoffDistr[idx]
                else:
                    L[V.index(avatar)] = 1.0
                
                U += theta[avatar] * L
            
            if np.sum(U) == 0:
                U = np.full_like(U, 1e-7)
            else:
                U /= np.sum(U)
            
            node_positions = [V.index(n) for n in node_order]
            U = U[node_positions]
            U[U == 0] = 1e-7
            ld = lossDistribution(U)
            payoffs.append(ld)
    return payoffs

def solve_game(payoffs, as1, as2):
    n = len(as1)
    m = len(as2)
    payoff_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            idx = i*m + j
            ld = payoffs[idx]
            payoff_matrix[i, j] = ld['dpdf'][-1]
    
    c = np.zeros(n+1)
    c[-1] = 1.0
    
    A_ub = np.zeros((m, n+1))
    for j in range(m):
        for i in range(n):
            A_ub[j, i] = payoff_matrix[i,j]
        A_ub[j, -1] = -1.0
    
    b_ub = np.zeros(m)
    A_eq = np.zeros((1, n+1))
    A_eq[0,:n] = 1.0
    b_eq = np.array([1.0])
    bounds = [(0,None)]*n + [(None,None)]
    
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)
    if res.success:
        x = res.x[:n]
        v = res.x[-1]
        optimal_defense = dict(zip(as1, x))
        return {
            'optimal_defense': optimal_defense,
            'attacker_success': v
        }
    else:
        print("LP did not converge:", res.message)
        return None


In [8]:
def run_game(graph, attack_rate_list, defense_rate_list, random_steps_fn):
    # Runs the game and logs results like R code does
    entry_node_candidates = [n for n,deg in graph.in_degree() if deg==0]
    if len(entry_node_candidates) != 1:
        print("WARNING: Not a single entry node found.")
    entry_node = entry_node_candidates[0]

    routes, V, as1, as2, target_list, node_order = generate_game_elements(graph, entry_node)
    adv_list, theta, m = setup_game_parameters(V, routes, entry_node, target_list)
    
    if not defense_rate_list:
        defense_rate_list = [0]
    if not attack_rate_list:
        attack_rate_list = [0]
    
    for defenseRate in defense_rate_list:
        for attackRate in attack_rate_list:
            # Instead of just printing to console, log to the file
            logger.info("\n++++++++++++++++++++++++++++++++")
            logger.info(f"attack rate =  {attackRate} , defense rate =  {defenseRate} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")
            
            payoffs = calculate_payoff_distribution(
                graph, as1, as2, V, adv_list, theta, random_steps_fn, attackRate, defenseRate, node_order
            )
            
            eq = solve_game(payoffs, as1, as2)
            if eq is not None:
                # Log optimal defense strategy like in R
                logger.info("optimal defense strategy:")
                logger.info("         prob.")
                # Sort keys if you want them ordered
                for node, prob in sorted(eq['optimal_defense'].items(), key=lambda x: str(x[0])):
                    logger.info(f"{node} {prob:.6e}")
                
                logger.info("\nworst case attack strategies per goal:")
                logger.info("          1")
                logger.info(f"1 {eq['attacker_success']:.7f}")
                logger.info(f"[1] {eq['attacker_success']:.3f}")
            else:
                logger.info("No solution found.")

In [ ]:
# Now run the game on work_graph 
run_game(work_graph, attack_rate_list=attack_rate_list, defense_rate_list=defense_rate_list, random_steps_fn=random_steps)

NameError: name 'attack_rate_list' is not defined